# Visão agrupada da URA x DERIVAÇÕES

**Objetivo:** consolidar chamadas da URA `tabe_ura_anlt`) e chamadas derivadas (`tabe_calls`), **D‑1** em uma visão diária por fila, materializada em `prd.g_dm_callcenter.visao_ura_calls` com `replaceWhere` por `DT_REFE`.

## Imports


In [0]:
from pyspark.sql import functions as F, Window as W
from datetime import datetime, timedelta

## Parâmetros


In [0]:
# Data de referência (D‑1 por padrão)
odate = (datetime.now() - timedelta(days=1)).date()

# Tabelas origens
nm_tabe_calls = 'prd.s_dm_callcenter.tabe_calls'
nm_tabe_ura_anlt  = 'prd.s_dm_callcenter.tabe_ura_anlt'

# Tabela final
nm_tabe_final = 'prd.g_dm_callcenter.visao_ura_calls'

## 1) URA

In [0]:
w_ura = W.partitionBy('ID_CHAM').orderBy(F.col('DH_INIC').asc())

df_tabe_ura = (
    # Base origem
    spark.table(nm_tabe_ura_anlt)

    .filter(F.col('DT_INIC') == odate)

    .withColumn('DT_REFE', F.to_date('DT_INIC')) 
    .withColumn('QT_OPCA_NAVG_LAST', F.last('QT_OPCA_NAVG', True).over(w_ura))
    .withColumn('IN_DERV_ATEN_ANY', F.max(F.col('IN_DERV_ATEN').cast('int')).over(w_ura))
    .withColumn('IN_AUTN_ANY',      F.max(F.col('IN_AUTN').cast('int')).over(w_ura))
    .select('ID_CHAM','ID_FILA','CD_PERI','DT_REFE','QT_OPCA_NAVG_LAST','IN_DERV_ATEN_ANY','IN_AUTN_ANY')
)

## 2) Chamadas


In [0]:
df_tabe_calls = (
    spark.table('prd.s_dm_callcenter.tabe_calls')
    .filter(F.col('DT_INIC') == odate) 
    .withColumn('TEMPO_CHAMADA', F.unix_timestamp('DH_FIM') - F.unix_timestamp('DH_INIC'))
    .groupBy('ID_CHAM')
    .agg(
        F.count('*').alias('QT_CHAM_ATEN_HUMN'),
        F.sum(F.coalesce(F.col('IN_TRAF'), F.lit(0))).alias('QT_TRAF'),
        F.sum(F.coalesce(F.col('IN_TRAF_INDV'), F.lit(0))).alias('QT_TRAF_INDV'),
        F.sum('TEMPO_CHAMADA').alias('NR_TEMP_CHAM')
    )
)

## 3) Join + Métricas agregadas

In [0]:
df_join = (
    df_tabe_ura.alias('ura')
    .join(df_tabe_calls.alias('c'), 'ID_CHAM', 'left')
    .withColumn('QT_CHAM_ATEN_HUMN', F.coalesce(F.col('QT_CHAM_ATEN_HUMN'), F.lit(0)))
    .withColumn('QT_TRAF',           F.coalesce(F.col('QT_TRAF'), F.lit(0)))
    .withColumn('QT_TRAF_INDV',      F.coalesce(F.col('QT_TRAF_INDV'), F.lit(0)))
    .withColumn('NR_TEMP_CHAM',      F.coalesce(F.col('NR_TEMP_CHAM'), F.lit(0.0)))

    .groupBy('CD_PERI', 'DT_REFE', 'ID_FILA')
    .agg(
        F.round(F.avg('QT_OPCA_NAVG_LAST'), 2).alias('NR_MED_OPCA_NAVG'),
        F.round(F.avg(F.when(F.col('IN_DERV_ATEN_ANY') == 0, F.col('QT_OPCA_NAVG_LAST'))), 2).alias('NR_MED_OPCA_AUTO_SERV'),
        F.round(F.avg(F.when(F.col('IN_DERV_ATEN_ANY') == 1, F.col('QT_OPCA_NAVG_LAST'))), 2).alias('NR_MED_OPCA_DERV'),
        F.sum('IN_AUTN_ANY').alias('QT_AUTN'),
        F.sum('IN_DERV_ATEN_ANY').alias('QT_DERV_ATEN'),
        F.sum('QT_CHAM_ATEN_HUMN').alias('QT_CHAM_ATEN_HUMN'),
        F.sum('QT_TRAF').alias('QT_TRAF'),
        F.sum('QT_TRAF_INDV').alias('QT_TRAF_INDV'),
        F.round(
            F.when(F.sum('QT_CHAM_ATEN_HUMN') > 0, F.sum('NR_TEMP_CHAM') / F.sum('QT_CHAM_ATEN_HUMN'))
            .otherwise(F.lit(None)), 2
        ).alias('NR_MEDI_TEMP_CHAM')
    )
)

## 4) Criação da tabela destino
Cria a tabela e aplica `CLUSTER BY (CD_PERI, DT_REFE, ID_ASST)`

In [0]:
if not spark.catalog.tableExists('prd.g_dm_callcenter.visao_ura_calls'):
    empty = spark.createDataFrame([], df_join.schema)
    (empty.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable('prd.g_dm_callcenter.visao_ura_calls'))
    spark.sql(f"ALTER TABLE {'prd.g_dm_callcenter.visao_ura_calls'} CLUSTER BY (CD_PERI, DT_REFE, ID_FILA)")

df_join.write \
    .mode('overwrite') \
    .option('replaceWhere', f"DT_REFE = '{odate}'") \
    .saveAsTable('prd.g_dm_callcenter.visao_ura_calls')